In [ ]:
MARKET = 'SNP'

In [ ]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [ ]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [ ]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [ ]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID
PAPER = ibp.PAPER

In [ ]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

In [ ]:
df_ch = pd.concat(
        (df_chains.assign(right="P"), df_chains.assign(right="C")), ignore_index=True
    )

In [ ]:
# ... build the options
opts = [
    Option(s, e, k, r, x)
    for s, e, k, r, x in zip(
        df_ch.symbol,
        df_ch.expiry,
        df_ch.strike,
        df_ch.right,
        ["NSE" if MARKET.upper() == "NSE" else "SMART"] * len(df_ch),
    )
]

In [ ]:
BLK_SIZE=1000
raw_blks = [opts[i:i + BLK_SIZE]
            for i in range(0, len(opts), BLK_SIZE)]

In [ ]:
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

In [ ]:
from tqdm.notebook import trange, tqdm

In [ ]:
%%time
qopts = set()

with IB().connect(HOST, PAPER, CID) as ib:
    for b in tqdm(raw_blks, desc='raw blocks'):
        qs = ib.qualifyContracts(*b)
        qopts.update(set(qs))  

In [ ]:
len(qopts), len(qs), len(opts)

In [ ]:
548236/714/10/60

In [ ]:
s = set([1, 2, 3])
s1 = set([4, 5, 6])
s.update(s1)

In [ ]:
s

In [ ]:
s1

In [7]:
import pandas as pd
pd.Series(list(set([1, 2, 3]))).to_pickle('temp.pkl')

In [3]:
set([1, 2, 3])

{1, 2, 3}

In [5]:
import pandas as p
s = {12,34,78,100}
df = p.DataFrame(list(s))
print(df)

     0
0  100
1   34
2   12
3   78


In [8]:
pd.read_pickle('temp.pkl')

0    1
1    2
2    3
dtype: int64